# Neutral simulations (Figure 1D)

## Import libraries and load data

In [ ]:
import numpy as np
import module
import plot
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
from matplotlib.offsetbox import AnchoredOffsetbox, TextArea, HPacker
import pickle
import itertools
import re
import json

Load experimental data from file

In [ ]:
dataset = pd.read_excel("expdata/plasmid-exp.xlsx", header=0)
dataset.head()

Get carrying capacities

In [ ]:
Nc=pd.read_csv('expdata/Nc_plasmid.csv', index_col=0).squeeze()
print('Carrying capacities loaded')

In [ ]:
Nc_all=[]
for i in range(6):
    Nc=np.array(
        dataset.loc[(dataset['replicate']==1 
        * (dataset['replicate.n']==i+1)),['total_pop']])[:,0]
    Nc_all.append(Nc)

D=min([len(Nc_all[i]) for i in range(len(Nc_all))])-1
print('Carrying capacities loaded: (day=1-'+str(D)+',rep1,..,rep6)')

t=np.array(range(1,D+1))
Nc_all=[Nc_all[i][1:D+1] for i in range(len(Nc_all))]

np.set_printoptions(precision=3)
print(np.vstack((t,Nc_all)).T)


## Simulation

Initialise parameter sets:  
10 simulations with regular replication  
10 simulations with random replication 

In [ ]:
path="./"
rep_list=["reg","ran"]
seed_list=list(range(30))
f_list=[10**(-4)]
l=list(itertools.product(rep_list,f_list,seed_list))
def parset(par):
    rep,f,seed=par
    return {"rep":rep,"seed":seed,"f":f,
                "Nc":Nc_all[seed%6][:].tolist(),
            "file":path+"neutral_data/"+re.sub('[^a-zA-Z0-9_]','_',
                '_'.join(map(str,
                ("rep",rep,"seed",seed)))
        )}
parsets=list(map(parset,l))
print(len(parsets), "parameter combinations initialised." )
print("First parameter combination example:" )
parsets[0]

Test simulation for single parameter combination

In [ ]:
parset=parsets[0]
print(parset)
sim=module.stochbottleSim(rwt=1.,n=15,D=30,d0=1,swb=False,**parset,verbose=True)
plot.plot_interdays(sim);

Run simulation for all other parameter combinations 

In [ ]:
# runtime ca 2 min
for i in range(1,60):
    sim=module.stochbottleSim(rwt=1.,n=15,D=30,d0=1,swb=False,**parsets[i])

Load simulation results from files 

In [ ]:
def parsetwoseedfile(par):
    rep=par[0]
    f=par[1]
    return {"rep":rep,"f":f}

l=list(itertools.product(rep_list,f_list))
parsetswoseedfile=list(map(parsetwoseedfile,l))

def loadparsetsim(parset):
    fn=parset["file"]
    #print(fn)
    try:
        sim=pickle.load(open(fn,'rb+'))

    except:
        print('file not found', fn)
        sim=None
    return sim

parameters=[{k:parsets[i][k] for k in parsets[i] if k!='seed' and k!='file' and k!='Nc' and k!='Npop'} for i in range(len(parsets))]
parameters
parsetswoseedfile

sims_allpar=[]

for i in range(len(parsetswoseedfile)):
    parameter=parsetswoseedfile[i]
    sims=[loadparsetsim(parsets[i]) 
            for i in range(len(parsets)) if parameters[i]==parameter]
    sims=[sims[i] for i in range(len(sims)) if sims[i]!=None]
    sims_allpar.append(sims)
    print("parameters: ", parameter," ", len(sims), " simulations loaded")


Plot all simulation replicates 

In [ ]:
for i in range(len(parsetswoseedfile)):
    parameter=parsetswoseedfile[i]
    sims=sims_allpar[i]

    title=json.dumps(parameter)
    fig,ax=plt.subplots()
    for i in range(len(sims)):
        plot.plot_interdays(sims[i],figax=(fig,ax),title=title+', nsim'+str(len(sims)) )

    title=re.sub('[^a-zA-Z0-9]','_',title)
    title=re.sub('_{2,}','_',title)
    print(title,end=', ')
    print(str(len(sims))+' simulations found.')



## Results

In [ ]:
# define function to load time series of experimental data
def gettimeseries_exp(replicate):
    data_reps=[]

    for ir in range(1,6+1):
        data = dataset.loc[(dataset['replicate']==replicate),['t','homo_freq','hetero_freq','replicate.n','host_freq']]
        data
        data_rep = data.loc[((data['replicate.n']==ir) ),['t','homo_freq','hetero_freq','host_freq']]
        data_reps.append(data_rep)

    return replicate,data_reps

Plot comparison of mean simulation data and experimental replicate data (Figure 1D)

In [ ]:
matplotlib.rcParams.update({'font.size': 8})

colorgreen='#008080' # colors from Fig 1A GFP (green) and nptII (red) respectively
colorred='#c83771'
alphared=0.5

fig,ax=plt.subplots(figsize=(7.5/2.54,5/2.54))
ax.set_position(pos=(0.23,0.2,0.75,0.75))
replicate,data_reps=gettimeseries_exp(1)
hetero_freq=[]
for i in range(6):
    plt.plot(data_reps[i]["t"][1:],data_reps[i]["homo_freq"][1:],'.-',fillstyle='full',color=colorred,alpha=.3*alphared,linewidth=.4,markersize=1, zorder=-1)
    hetero_freq.append(np.array(data_reps[i]["hetero_freq"][:]))
    plt.plot(data_reps[i]["t"][1:],data_reps[i]["hetero_freq"][1:],'.-',fillstyle='full',color=colorgreen,alpha=.3*1,linewidth=.4,markersize=1, zorder=1)
hetero_freq_median=np.median(hetero_freq,axis=0)

ls=['solid','dotted']  
cl=[colorgreen,colorgreen]
clhom=[colorred,colorred]
zo=[0,-1]

for i in np.array(range(len(parsetswoseedfile)))[[1,0]]:#: 
    parameter=parsetswoseedfile[i]

    ts_eod=np.array([sims_allpar[i][j][1] for j in range(len(sims_allpar[i]))])
    ts_eod_het_freq=np.sum(ts_eod[:,:,1:-1], axis=-1)/np.sum(ts_eod,axis=-1)
    ts_eod_het_freq_median=np.median(ts_eod_het_freq,axis=0)
    q_het=np.quantile(ts_eod_het_freq,q=[.25,.75], axis=0)

    ts_eod_hom_freq=np.sum(ts_eod[:,:,-1:], axis=-1)/np.sum(ts_eod,axis=-1)
    ts_eod_hom_freq_median=np.median(ts_eod_hom_freq,axis=0)
    q_hom=np.quantile(ts_eod_hom_freq,q=[.25,.75], axis=0)

    label=''
    plt.plot(range(1,ts_eod.shape[1]+1),ts_eod_hom_freq_median,
        color=clhom[i],fillstyle='none',linestyle=ls[i],linewidth=2, marker=None,alpha=alphared,label=label, zorder=zo[i])
    plt.plot(range(1,ts_eod.shape[1]+1),ts_eod_het_freq_median,
        color=cl[i],fillstyle='none',linestyle=ls[i],linewidth=2, marker=None,alpha=1.,label=label)
    
    plt.fill_between(range(1,len(q_het[0])+1),y1=q_het[0],y2=q_het[-1],
        color=cl[i],alpha=.2,edgecolor="none")
    plt.fill_between(range(1,len(q_hom[0])+1),y1=q_hom[0],y2=q_hom[-1],
        color=clhom[i],alpha=.2,edgecolor="none")


plt.yscale('log')
lim=(1e-8*0.95,1e-4/0.95);plt.ylim(lim)
lim=(+.5,30.5);plt.xlim(lim)
plt.xticks([1,5,10,15,20,25,30])
plt.yticks([1e-04,1e-05,1e-06,1e-07,1e-08])
plt.xlabel('Number of transfer', fontsize = 8.0)

ylabel_het = TextArea("Heterozygote frequency ", textprops=dict(color=colorgreen, size=8.0,rotation=90,ha='left',va='bottom'))
ylabel_hom = TextArea("Homozygote frequency ", textprops=dict(color=colorred, alpha=alphared, size=8.0,rotation=90,ha='left',va='bottom'))
ybox = HPacker(children=[ylabel_het, ylabel_hom],align="center", pad=0, sep=2)
anchored_ybox = AnchoredOffsetbox(loc=8, child=ybox, pad=0., frameon=False, bbox_to_anchor=(-0.24, 0.0), bbox_transform=ax.transAxes, borderpad=0.)
ax.add_artist(anchored_ybox)

plt.savefig('../figures-plots/plot_1D.pdf')
